# Playlist testing
Again roughly following
https://stmorse.github.io/journal/spotify-api.html 

In [223]:
import requests
import pandas as pd


# Taken from Spotify App Dashboard, secret may auto-rotate.
CLIENT_ID = "324585f433a84680a7a7a6414c721d13"
CLIENT_SECRET = "7125b0da4a57479b91d97102d505c74d"

In [224]:
AUTH_URL = "https://accounts.spotify.com/api/token"

# Post request for client credentials
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET
    })

# Convert response to json
auth_response_data = auth_response.json()

# Save the access token
access_token = auth_response_data['access_token']

print(auth_response_data)
# Note: Access token expires in 3600sec? = 1 hour?

{'access_token': 'BQB9Xy7v9bxZUAlCCPhdKJYC2fcYtbRKmHTGPbYXc5SymOKItX85iw5IKADmvTsVB1FIpKM5PmTJ-Aqw8jQ', 'token_type': 'Bearer', 'expires_in': 3600}


In [225]:
headers = {
    'Authorization' : 'Bearer {token}'.format(token=access_token)
}

# Testing out with single file

In [4]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

# Track ID for Classic-MKTO
# https://open.spotify.com/track/6FE2iI43OZnszFLuLtvvmg?si=e4e75d9c000a4219
track_id = '6FE2iI43OZnszFLuLtvvmg'

# Make GET request using header for access token
r = requests.get(BASE_URL + 'audio-features/'+track_id, headers=headers)
r=r.json()

In [5]:
display(r)
norm_params = ['danceability', 'energy', 'speechiness', 'acousticness',\
               'instrumentalness', 'liveness', 'valence']

extra_params = ['tempo', 'mode', 'duration']

for i in norm_params:
    print("%s = %.4f" %(i, r[i]))

{'danceability': 0.72,
 'energy': 0.791,
 'key': 1,
 'loudness': -4.689,
 'mode': 1,
 'speechiness': 0.124,
 'acousticness': 0.0384,
 'instrumentalness': 0,
 'liveness': 0.157,
 'valence': 0.756,
 'tempo': 102.071,
 'type': 'audio_features',
 'id': '6FE2iI43OZnszFLuLtvvmg',
 'uri': 'spotify:track:6FE2iI43OZnszFLuLtvvmg',
 'track_href': 'https://api.spotify.com/v1/tracks/6FE2iI43OZnszFLuLtvvmg',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6FE2iI43OZnszFLuLtvvmg',
 'duration_ms': 175427,
 'time_signature': 4}

danceability = 0.7200
energy = 0.7910
speechiness = 0.1240
acousticness = 0.0384
instrumentalness = 0.0000
liveness = 0.1570
valence = 0.7560


# Now working on playlist

In [156]:
# Classic Rock (300 songs)
#https://open.spotify.com/playlist/3Ho3iO0iJykgEQNbjB2sic?si=4c64e34eac34483a
#last /*? is the id
playlist_name='ClassicRock'
playlist_id = '3Ho3iO0iJykgEQNbjB2sic'

offset=0
limit=100
data = [] # will hold all track info
for i in range(3):
    r = requests.get(BASE_URL + 'playlists/' + playlist_id + '/tracks',
                    headers=headers,
                    params={'limit': limit, 'offset': offset})
    offset+=limit

    d = r.json()
    print("i=%d successful"%i)
    ######

    for track in d['items']:
        # get audio features
        f = requests.get(BASE_URL + 'audio-features/'+track['track']['id'],
                        headers=headers)

        f=f.json()

        # combine features json with album info
        f.update({
            'track_name': track['track']['name'],
        })

        data.append(f)



df = pd.DataFrame(data)
df.to_csv("../SpotifyData/"+playlist_name + "_" + playlist_id+".csv")

i=0 successful
i=1 successful
i=2 successful


In [155]:
df.to_csv("../SpotifyData/"+playlist_name + "_" + playlist_id+".csv")

# Putting this in to a function


In [228]:
def get_playlist_song_parameters(playlist_name, playlist_id, playlist_length):
    #Adjust limit+offset to max out at playlist_length
    offset=0
    limit=100
    data = [] # will hold all track info
    
    print("Current song count = %d of %d" %(offset, playlist_length))
    while (offset+limit <= playlist_length):
        r = requests.get(BASE_URL + 'playlists/' + playlist_id + '/tracks',
                        headers=headers,
                        params={'limit': limit, 'offset': offset})
        
        display(r)
        offset+=limit
        if (offset+limit>playlist_length):
            limit = playlist_length-offset
            
        print("Current song count = %d of %d" %(offset, playlist_length))
        d = r.json()

        for track in d['items']:
            # get audio features
            f = requests.get(BASE_URL + 'audio-features/'+track['track']['id'],
                            headers=headers)

            f=f.json()

            # combine features json with album info
            f.update({
                'track_name': track['track']['name'],
            })

            data.append(f)

        if limit==0:
            break
        
    df = pd.DataFrame(data)
    df.to_csv("../SpotifyData/"+playlist_name + "_" + playlist_id+".csv")
    
#end function

In [227]:
"""
# Classic Rock (300 songs)
#https://open.spotify.com/playlist/3Ho3iO0iJykgEQNbjB2sic?si=4c64e34eac34483a
#last /*? is the id
playlist_name='ClassicRock'
playlist_id = '3Ho3iO0iJykgEQNbjB2sic'
playlist_length=300
get_playlist_song_parameters(playlist_name, playlist_id, playlist_length)

# Jazz Classic (200 songs)
#https://open.spotify.com/playlist/37i9dQZF1DXbITWG1ZJKYt?si=7183171d6bda41d7
playlist_name='JazzClassic'
playlist_id='37i9dQZF1DXbITWG1ZJKYt'
playlist_length=200
get_playlist_song_parameters(playlist_name, playlist_id, playlist_length)

# CountryHits (206 songs)
#https://open.spotify.com/playlist/1mJhSx6aYQmINsZ8dG4gzU?si=64b34f14280e42fb
playlist_name='CountryHits'
playlist_id='1mJhSx6aYQmINsZ8dG4gzU'
playlist_length=206
get_playlist_song_parameters(playlist_name, playlist_id, playlist_length)
"""
# DanceHits (433 songs)
#https://open.spotify.com/playlist/5oKz4DsTP8zbL97UIPbqp4?si=7d07c97ae6b441d1
playlist_name='DanceHits'
playlist_id='5oKz4DsTP8zbL97UIPbqp4'
playlist_length=433
get_playlist_song_parameters(playlist_name, playlist_id, playlist_length)
"""This one breaks^ unsure why"""

# Hiphop (230 songs)
#https://open.spotify.com/playlist/5CRJExGSQFYXem2hT5GFkd?si=88aa754b7aa4426f
playlist_name='HipHopClassics'
playlist_id='5CRJExGSQFYXem2hT5GFkd'
playlist_length=230
get_playlist_song_parameters(playlist_name, playlist_id, playlist_length)

# ClassicalStudy (228 songs)
#https://open.spotify.com/playlist/6wObnEPQ63a4kei1sEcMdH?si=9bcb78a6bbcd46c9
playlist_name='ClassicalStudy'
playlist_id='6wObnEPQ63a4kei1sEcMdH'
playlist_length=228
get_playlist_song_parameters(playlist_name, playlist_id, playlist_length)

# Heavy Metal Classics (386 songs)
#https://open.spotify.com/playlist/27gN69ebwiJRtXEboL12Ih?si=10ae3181e8fa4348
playlist_name='HeavyMetalClassics'
playlist_id='27gN69ebwiJRtXEboL12Ih'
playlist_length=386
get_playlist_song_parameters(playlist_name, playlist_id, playlist_length)

# Kpop Classics (323 songs)
#https://open.spotify.com/playlist/0jB4ANR4ox65etDMnxvGLp?si=12bd73e2ea354ca7
playlist_name='KpopClassics'
playlist_id='0jB4ANR4ox65etDMnxvGLp'
playlist_length=323
get_playlist_song_parameters(playlist_name, playlist_id, playlist_length)

Current song count = 0 of 433


<Response [200]>

<Response [200]>

<Response [200]>

<Response [200]>

...

Current song count = 0 of 323


<Response [200]>

<Response [200]>

<Response [200]>

<Response [200]>

In [229]:
#for track in d['items']:
 #   print(track['track']['name'])